In [4]:
from decimal import Decimal, getcontext
from typing import List, Dict
import threading
import time

# Use Decimal for money to avoid floating-point rounding errors
getcontext().prec = 28

class InsufficientFunds(Exception):
    """Raised when a withdrawal would exceed the allowed balance (and overdraft not allowed)."""
    pass

class BankAccount:
    """
    Simple BankAccount supporting deposit, withdraw and balance retrieval.
    - Uses Decimal for money.
    - Thread-safe via a lock.
    - Keeps a small transaction history for auditing (in-memory).
    """

    def __init__(self, owner: str, initial_balance: Decimal = Decimal('0.00'), allow_overdraft: bool = False):
        if initial_balance < 0:
            raise ValueError("initial_balance cannot be negative")
        self.owner = owner
        self._balance = Decimal(initial_balance)
        self.allow_overdraft = bool(allow_overdraft)
        self._lock = threading.Lock()
        self._txn_history: List[Dict] = []
        self._record_txn("INIT", Decimal(initial_balance))

    def _record_txn(self, ttype: str, amount: Decimal) -> None:
        """Append a timestamped transaction record."""
        self._txn_history.append({
            "timestamp": time.time(),
            "type": ttype,
            "amount": Decimal(amount),
            "balance_after": Decimal(self._balance)
        })

    def deposit(self, amount: Decimal) -> Decimal:
        """
        Add a positive amount to the balance and return the new balance.
        Raises ValueError for non-positive amounts.
        Thread-safe.
        """
        amt = Decimal(amount)
        if amt <= 0:
            raise ValueError("Deposit amount must be positive")
        with self._lock:
            self._balance += amt
            self._record_txn("DEPOSIT", amt)
            return Decimal(self._balance)

    def withdraw(self, amount: Decimal) -> Decimal:
        """
        Subtract a positive amount from the balance and return the new balance.
        Raises:
          - ValueError for non-positive amounts
          - InsufficientFunds if overdraft not allowed and balance insufficient
        Thread-safe.
        """
        amt = Decimal(amount)
        if amt <= 0:
            raise ValueError("Withdrawal amount must be positive")
        with self._lock:
            if not self.allow_overdraft and self._balance - amt < 0:
                raise InsufficientFunds("Insufficient funds for withdrawal")
            self._balance -= amt
            self._record_txn("WITHDRAW", amt)
            return Decimal(self._balance)

    def get_balance(self) -> Decimal:
        """Return current balance (Decimal)."""
        with self._lock:
            return Decimal(self._balance)

    def get_transaction_history(self) -> List[Dict]:
        """Return a shallow copy of transaction history for auditing."""
        with self._lock:
            return list(self._txn_history)

    def __repr__(self) -> str:
        return f"<BankAccount owner={self.owner!r} balance={self._balance:.2f}>"

In [5]:
from decimal import Decimal

acc = BankAccount("Priya", initial_balance=Decimal('100.00'))
print(acc)                       # <BankAccount owner='Priya' balance=100.00>
print(acc.deposit(Decimal('50.00')))   # 150.00
print(acc.withdraw(Decimal('30.00')))  # 120.00
print(acc.get_balance())          # Decimal('120.00')
print(acc.get_transaction_history())

<BankAccount owner='Priya' balance=100.00>
150.00
120.00
120.00
[{'timestamp': 1762624798.1259177, 'type': 'INIT', 'amount': Decimal('100.00'), 'balance_after': Decimal('100.00')}, {'timestamp': 1762624798.1262755, 'type': 'DEPOSIT', 'amount': Decimal('50.00'), 'balance_after': Decimal('150.00')}, {'timestamp': 1762624798.126389, 'type': 'WITHDRAW', 'amount': Decimal('30.00'), 'balance_after': Decimal('120.00')}]


In [6]:
def test_bank_account_basic():
    from decimal import Decimal
    a = BankAccount("T", initial_balance=Decimal('100.00'))
    assert a.get_balance() == Decimal('100.00')
    a.deposit(Decimal('25.00'))
    assert a.get_balance() == Decimal('125.00')
    a.withdraw(Decimal('25.00'))
    assert a.get_balance() == Decimal('100.00')
    import pytest
    with pytest.raises(InsufficientFunds):
        a.withdraw(Decimal('1000.00'))
    with pytest.raises(ValueError):
        a.deposit(Decimal('0.00'))